In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
from bigru import Encoder,Decoder,Seq2Seq
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 32 # butuh lebih banyak update 
EPOCHS=20
DATA_PATH = "dataset/"  
SAVE_DIR = "saved"
train_loader, val_loader, test_loader = get_dataloaders(
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)
SRC_VOCAB_SIZE = 4000  
TGT_VOCAB_SIZE = 4000  
EMBED_SIZE = 256
ENC_HIDDEN = 64    
DEC_HIDDEN = ENC_HIDDEN*2 #2 KALI KARENA DARI BIGRU EMBEDDINGNYA 2X   [x_1 -> x_2 ] cat [x_1 <- x_2 ]    
N_LAYERS = 2
DROP_OUT = 0.2
encoder = Encoder(SRC_VOCAB_SIZE, EMBED_SIZE, ENC_HIDDEN, num_layers=N_LAYERS, dropout=DROP_OUT, pad_idx=utils.PAD_TOKEN)
decoder = Decoder(TGT_VOCAB_SIZE, EMBED_SIZE, DEC_HIDDEN, num_layers=N_LAYERS, dropout=DROP_OUT, pad_idx=utils.PAD_TOKEN)
model = Seq2Seq(encoder, decoder, device, ENC_HIDDEN, DEC_HIDDEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)

TrainData - Max 'min' sentence length: 76
TrainData - Max 'eng' sentence length: 107
TestData - Max 'min' sentence length: 61
TestData - Max 'eng' sentence length: 75
ValidData - Max 'min' sentence length: 62
ValidData - Max 'eng' sentence length: 81
Number of examples in train_dataset,train origin,train_raw: 799 799 799
Number of examples in valid_dataset: 100
Number of examples in test_dataset: 100
Model initialized on: cuda


In [3]:
tokens = {
    "Padding": utils.PAD_TOKEN,
    "Start of Sequence": utils.SOS_TOKEN,
    "End of Sequence": utils.EOS_TOKEN,
    "Unknown": utils.UNK_TOKEN
}
for i, batch in enumerate(train_loader):
    if i < 8:
        src = batch["src"]
        tgt = batch["tgt"]
        ss,fss=src[0,:],src.shape
        st,fst=tgt[0,:],tgt.shape
        print(fss,fst)
    for name, token in tokens.items():
        print(f"{name}: {token}")
    else:
        break

torch.Size([32, 109]) torch.Size([32, 109])
Padding: 0
Start of Sequence: 1
End of Sequence: 2
Unknown: 3


testing input , is it correct

In [4]:
data_iter = iter(train_loader)
for i in range(7):
    batch = next(data_iter)
    print(batch["src"].shape)


torch.Size([32, 109])
torch.Size([32, 109])
torch.Size([32, 109])
torch.Size([32, 109])
torch.Size([32, 109])
torch.Size([32, 109])
torch.Size([32, 109])


In [5]:
for i,batch in enumerate(train_loader):
    if i <1:
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        output,_= model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        print(tgt_y)

tensor([1038,  190,  851,  ...,    0,    0,    0], device='cuda:0')


In [6]:
import os
import torch
from tqdm import tqdm
os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf") 
best_model_path = None
atl = {}
avl = {}
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=True, total=len(train_loader))
    for batch in train_bar:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)

        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    atl[epoch]=avg_train_loss

    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    avl[epoch]=avg_val_loss

    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best_gru.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")
utils.plot_loss(atl, avl, SAVE_DIR,"loss_bigru")

Epoch 1/20


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 27.12it/s, loss=7.1819]


[Epoch 1] Train Loss: 7.9795 | Val Loss: 7.2310
  -> New best model saved at saved/best_gru.pt
Epoch 2/20


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 27.27it/s, loss=6.5234]


[Epoch 2] Train Loss: 6.6391 | Val Loss: 6.5592
  -> New best model saved at saved/best_gru.pt
Epoch 3/20


...

[Epoch 18] Train Loss: 6.1383 | Val Loss: 6.7332
Epoch 19/20


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 19.79it/s, loss=6.7300]


[Epoch 19] Train Loss: 6.1224 | Val Loss: 6.7321
Epoch 20/20


🚀 Validation: 100%|██████████| 4/4 [00:00<00:00, 26.98it/s, loss=6.7168]


[Epoch 20] Train Loss: 6.1076 | Val Loss: 6.7244
Loss plot saved to saved/loss_bigru.png
